# Import libraries

In [1]:
import numpy as np 
import torch
import torch.nn.functional as F 
from types import FunctionType
from typing import Tuple

# Different implementations of the convolution function

In [2]:
class CustomConvolution():
    def __init__(self, input_data: torch.Tensor, filter_weigts: torch.Tensor, bias: torch.Tensor, stride: int = 1, padding: int = 0) -> None:
        self.__input_data = input_data
        self.__channels_input, self.__height_input, self.__width_input = self.__input_data.shape
        self.__filter_weigts = filter_weigts
        self.__numbers_filters, self.__channels_conv, self.__height_conv, self.__width_conv = self.filter_weigts.shape 
        self.__bias = bias
        self.__stride = stride
        self.__padding = padding
    
    @property
    def input_data(self,) -> torch.Tensor:
        return self.__input_data
    
    @property
    def input_data_shape(self,) -> Tuple[int, int, int]:
        return self.__channels_input, self.__height_input, self.__width_input

    @property
    def filter_weigts(self,) -> torch.Tensor:
        return self.__filter_weigts
    
    @property
    def filter_weigts_shape(self,) ->  Tuple[int, int, int, int]:
        return self.__numbers_filters, self.__channels_conv, self.__height_conv, self.__width_conv
    
    @property
    def bias(self,) -> torch.Tensor:
        return self.__bias
    
    @filter_weigts.setter
    def filter_weigts(self, new_filter_weigts: torch.Tensor) -> None:
        self.__filter_weigts = new_filter_weigts
        self.__numbers_filters, self.__channels_conv, self.__height_conv, self.__width_conv = self.filter_weigts.shape 

    @bias.setter
    def bias(self, new_bias) -> None:
        self.__bias = new_bias
    
    @property
    def stride(self,) -> int:
        return self.__stride
    
    @property
    def padding(self,) -> int:
        return self.__padding
    
    @stride.setter
    def stride(self, new_stride) -> None:
       self.__stride = new_stride
    
    @padding.setter
    def padding(self, new_padding) -> None:
       self.__padding = new_padding
    
    def __get_output_shape(self,) -> Tuple[int, int]:
        height = int((self.__height_input + 2 * self.__padding - (self.__height_conv - 1) - 1) / self.__stride + 1)
        width = int((self.__width_input + 2 * self.__padding - (self.__width_conv - 1) - 1) / self.__stride + 1)
        return height, width
    
    def __get_padding(self,) -> torch.Tensor:
         return F.pad(input=self.__input_data, pad=(self.__padding, self.__padding, self.__padding, self.__padding, 0, 0))
    
    def __check(self,):
        if self.__channels_input != self.__channels_conv:
            raise Exception(f"The number of channels of the source image - {self.__channels_input} and the number of convolution channels - {self.__channels_conv} do not coincide") 

    def conv_(self,) -> torch.Tensor:
        self.__check()
        input_data_padded = self.__get_padding()
        height_output, width_output = self.__get_output_shape()
        output_data = torch.zeros(size=(self.__numbers_filters, height_output, width_output), dtype=torch.float32) 

        for filter_index in range(self.__numbers_filters):
            for height_output_pixel_index in range(height_output):
                for width_output_pixel_index in range(width_output):
                    for height_conv_pixel_index in range(self.__height_conv):
                        for width_conv_pixel_index in range(self.__width_conv):
                            for channel_index in range(self.__channels_input):
                                output_data[filter_index][height_output_pixel_index][width_output_pixel_index] += input_data_padded[channel_index][self.__stride * height_output_pixel_index + height_conv_pixel_index][self.__stride * width_output_pixel_index + width_conv_pixel_index] * self.__filter_weigts[filter_index][channel_index][height_conv_pixel_index][width_conv_pixel_index]

                    output_data[filter_index][height_output_pixel_index][width_output_pixel_index] += self.__bias[filter_index]
        return output_data
        

In [3]:
class CustomConvolutionIm2Col():
    def __init__(self, input_data: torch.Tensor, filter_weights: torch.Tensor, bias: torch.Tensor, stride: int = 1, padding: int = 0) -> None:
        self.__input_data = input_data
        self.__channels_input, self.__height_input, self.__width_input = self.__input_data.shape
        self.__filter_weights = filter_weights
        self.__numbers_filters, self.__channels_conv, self.__height_conv, self.__width_conv = self.filter_weights.shape 
        self.__bias = bias
        self.__stride = stride
        self.__padding = padding
    
    @property
    def input_data(self,) -> torch.Tensor:
        return self.__input_data
    
    @property
    def input_data_shape(self,) -> Tuple[int, int, int]:
        return self.__channels_input, self.__height_input, self.__width_input

    @property
    def filter_weights(self,) -> torch.Tensor:
        return self.__filter_weights
    
    @property
    def filter_weights_shape(self,) ->  Tuple[int, int, int, int]:
        return self.__numbers_filters, self.__channels_conv, self.__height_conv, self.__width_conv
    
    @property
    def bias(self,) -> torch.Tensor:
        return self.__bias
    
    @filter_weights.setter
    def filter_weights(self, new_filter_weights: torch.Tensor) -> None:
        self.__filter_weights = new_filter_weights
        self.__numbers_filters, self.__channels_conv, self.__height_conv, self.__width_conv = self.filter_weights.shape 

    @bias.setter
    def bias(self, new_bias) -> None:
        self.__bias = new_bias
    
    @property
    def stride(self,) -> int:
        return self.__stride
    
    @property
    def padding(self,) -> int:
        return self.__padding
    
    @stride.setter
    def stride(self, new_stride) -> None:
       self.__stride = new_stride
    
    @padding.setter
    def padding(self, new_padding) -> None:
       self.__padding = new_padding
    
    def __get_output_shape(self,) -> Tuple[int, int]:
        height = int((self.__height_input + 2 * self.__padding - (self.__height_conv - 1) - 1) / self.__stride + 1)
        width = int((self.__width_input + 2 * self.__padding - (self.__width_conv - 1) - 1) / self.__stride + 1)
        return height, width
    
    def __get_padding(self,) -> torch.Tensor:
         return F.pad(input=self.__input_data, pad=(self.__padding, self.__padding, self.__padding, self.__padding, 0, 0))
    
    def __check(self,):
        if self.__channels_input != self.__channels_conv:
            raise Exception(f"The number of channels of the source image - {self.__channels_input} and the number of convolution channels - {self.__channels_conv} do not coincide") 

    def conv_(self,) -> torch.Tensor:
        self.__check()
        input_data_padded = self.__get_padding()
        height_output, width_output = self.__get_output_shape()

        output_data = torch.zeros(size=(self.__numbers_filters, height_output, width_output), dtype=torch.float32) 
        input_data_2d = torch.zeros(size=(self.__channels_conv * self.__height_conv * self.__width_conv, height_output * width_output), dtype=torch.float32)
        width_2d = torch.zeros(size=(self.__numbers_filters, self.__channels_conv * self.__height_conv * self.__width_conv), dtype=torch.float32) 

        for patch_index in range(height_output * width_output): 
            row_index = patch_index % width_output 
            column_index = patch_index // width_output
            input_data_2d[:, patch_index] = input_data_padded[:, self.__stride * row_index: self.__stride * row_index + self.__height_conv, self.__stride * column_index: self.__stride * column_index + self.__width_conv].flatten()
        
        for filter_index in range(self.__numbers_filters): 
            width_2d[filter_index] = self.__filter_weights[filter_index].flatten()

        output_data = torch.matmul(width_2d, input_data_2d) 
        for filter_index in range(self.__numbers_filters):
            output_data[filter_index] += self.__bias[filter_index] 
        
        # for testing
        output_data = output_data.reshape(self.__numbers_filters, height_output, width_output) 
        output_data = output_data.transpose(1, 2) 
        return output_data 


# Testing

In [4]:
class Test():
    def __init__(self, input_data, conv) -> None:
        self.__input_data = input_data
        self.__conv = conv

    def __get_bencmark(self, in_channels_: int, out_channels_: int, kernel_size_: int, stride_: int, padding_: int, weights: torch.Tensor, bias: torch.Tensor) -> torch.nn.Conv2d:
        conv = torch.nn.Conv2d(in_channels=in_channels_, out_channels=out_channels_, kernel_size=kernel_size_, stride=stride_, padding=padding_)
        conv.weight.data = weights 
        conv.bias.data = bias
        return conv
    
    def __get_filter(self, amount_filters: int, channels: int, kernel_size: int, low_: int = 0, high_: int = 10) -> Tuple[torch.Tensor, torch.Tensor]:
        filter_weights = torch.randint(low=low_, high=high_, size=(amount_filters, channels, kernel_size, kernel_size), dtype=torch.float32)
        bias_ = torch.randint(low=low_, high=high_, size=(amount_filters,), dtype=torch.float32)
        return filter_weights, bias_
    
    def test(self, parameters: dict) -> None:
        test_pass, test_fail = 0, 0
        channels_input, _, _ = self.__input_data.shape

        for amount_filters in parameters['numbers_filters']:
            for kernel_size in parameters['kernel_sizes']:
                for stride in parameters['stride']:
                    for padding in parameters['padding']:
                        filter_weights, bias_ = self.__get_filter(amount_filters, channels_input, kernel_size)
                        bencmark_conv = self.__get_bencmark(channels_input, amount_filters, kernel_size, stride, padding, filter_weights, bias_)
                        
                        bencmark_output = bencmark_conv(self.__input_data)
                        conv_ = self.__conv(self.__input_data, filter_weights, bias_, stride, padding)
                        func_conv_output = conv_.conv_()

                        if torch.equal(func_conv_output, bencmark_output):
                            test_pass += 1
                            outtext = 'pass'
                        else:
                            test_fail += 1
                            outtext = 'fail'
                            
                    print(f'Test {outtext}:\n - amount_filters: {amount_filters}\n - kernel_size: {kernel_size}\n - stride: {stride}\n - padding: {padding}\n')
        
        print(f"Number of tests passed: {test_pass}\nNumber of tests failed: {test_fail}")
        
        

In [5]:
# generated input
channels = torch.randint(1, 4, (1,))
shape_ = torch.randint(0, 513, (1,))
print(f'Shape input: [{channels[0]}, {shape_[0]}, {shape_[0]}]')
input_size = (channels, shape_, shape_)
generated_tensor_input = torch.randint(0, 256, input_size, dtype=torch.float32)

Shape input: [3, 261, 261]


In [6]:
parametrs_1 = {
    'numbers_filters': [1, 5],
    'kernel_sizes': [3, 5],
    'stride': [1, 2],
    'padding': [0, 2]
    }

test_covn_own = Test(generated_tensor_input, CustomConvolution) 
test_covn_own.test(parametrs_1) 

Test pass:
 - amount_filters: 1
 - kernel_size: 3
 - stride: 1
 - padding: 2

Test pass:
 - amount_filters: 1
 - kernel_size: 3
 - stride: 2
 - padding: 2

Test pass:
 - amount_filters: 1
 - kernel_size: 5
 - stride: 1
 - padding: 2

Test pass:
 - amount_filters: 1
 - kernel_size: 5
 - stride: 2
 - padding: 2

Test pass:
 - amount_filters: 5
 - kernel_size: 3
 - stride: 1
 - padding: 2

Test pass:
 - amount_filters: 5
 - kernel_size: 3
 - stride: 2
 - padding: 2

Test pass:
 - amount_filters: 5
 - kernel_size: 5
 - stride: 1
 - padding: 2

Test pass:
 - amount_filters: 5
 - kernel_size: 5
 - stride: 2
 - padding: 2

Number of tests passed: 16
Number of tests failed: 0


In [7]:
parametrs_2 = {
    'numbers_filters': [1, 5, 20],
    'kernel_sizes': [3, 5, 7, 16, 32],
    'stride': [1, 2, 3],
    'padding': [0, 1, 2, 4]
    }
test_covn_im2col = Test(generated_tensor_input, CustomConvolutionIm2Col) 
test_covn_im2col.test(parametrs_2) 

Test pass:
 - amount_filters: 1
 - kernel_size: 3
 - stride: 1
 - padding: 4

Test pass:
 - amount_filters: 1
 - kernel_size: 3
 - stride: 2
 - padding: 4

Test pass:
 - amount_filters: 1
 - kernel_size: 3
 - stride: 3
 - padding: 4

Test pass:
 - amount_filters: 1
 - kernel_size: 5
 - stride: 1
 - padding: 4

Test pass:
 - amount_filters: 1
 - kernel_size: 5
 - stride: 2
 - padding: 4

Test pass:
 - amount_filters: 1
 - kernel_size: 5
 - stride: 3
 - padding: 4

Test pass:
 - amount_filters: 1
 - kernel_size: 7
 - stride: 1
 - padding: 4

Test pass:
 - amount_filters: 1
 - kernel_size: 7
 - stride: 2
 - padding: 4

Test pass:
 - amount_filters: 1
 - kernel_size: 7
 - stride: 3
 - padding: 4

Test pass:
 - amount_filters: 1
 - kernel_size: 16
 - stride: 1
 - padding: 4

Test pass:
 - amount_filters: 1
 - kernel_size: 16
 - stride: 2
 - padding: 4

Test pass:
 - amount_filters: 1
 - kernel_size: 16
 - stride: 3
 - padding: 4

Test pass:
 - amount_filters: 1
 - kernel_size: 32
 - stride: